# Color Detection

In [1]:
import cv2
import numpy as np
# pip install Pillow
from PIL import Image

In [3]:
# function that gives a range on hues given a color
def get_limits(color):
    c = np.uint8([[color]])
    hsvC = cv2.cvtColor(c, cv2.COLOR_BGR2HSV)
    
    lowerLimit = hsvC[0][0][0] - 10, 100, 100
    upperLimit = hsvC[0][0][0] + 10, 255, 255
    # the +/-10 defines the range of hues that fall within the limits (the h in hsv)
    # the range on saturation and value is much bigger because we are only looking for hue
    
    lowerLimit = np.array(lowerLimit, dtype=np.uint8)
    upperLimit = np.array(upperLimit, dtype=np.uint8)

    return lowerLimit, upperLimit

In [5]:
yellow = [0, 255, 255] # color in BGR colorspace

capture = cv2.VideoCapture(0)
while True:
    ret, frame = capture.read()

    frame_blur = cv2.GaussianBlur(frame, (11, 11), 9)
    # blurring the image may help get the desired result, but it can be removed
    
    image_hsv = cv2.cvtColor(frame_blur, cv2.COLOR_BGR2HSV) # convert to HSV

    lowerLimit, upperLimit = get_limits(yellow) # range of hues that we want the software to detect
    
    mask = cv2.inRange(image_hsv, lowerLimit, upperLimit) # detects objects in color range

    '''mask2 = Image.fromarray(mask) # converting mask into Pillow format
    box = mask2.getbbox() # adds bounding boxes to mask
    if box is not None:
        x1, y1, x2, y2 = box
        cv2.rectangle(frame, (x1, y1), (x2, y2), (255, 0, 255), 5)'''

    contours, hierarchy = cv2.findContours(mask, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)

    for cnt in contours:
        if cv2.contourArea(cnt) > 100:  # only continues if size of the object is large enough (removes noise)
            x1, y1, w, h = cv2.boundingRect(cnt) # finds a bounding box for each object
            cv2.rectangle(frame, (x1, y1), (x1 + w, y1 + h), (0, 0, 255), 2)
    
    cv2.imshow('webcam', frame)
    cv2.imshow('blur', frame_blur)
    cv2.imshow('mask', mask)
    if cv2.waitKey(40) & 0xFF == ord(' '):
        break  # ends loop when spacebar is pressed

capture.release()
cv2.destroyAllWindows()  # closes window, only reaches here when spacebar is pressed